In [15]:
from datetime import datetime, date

from sqlalchemy import Column, DateTime, Date, String, Integer, func
from sqlalchemy.ext.declarative import declarative_base


def filter_func(val):
    if isinstance(val, datetime):
        return val.strftime('%Y-%m-%d %H:%M:%S')
    elif isinstance(val, date):
        return val.strftime('%Y-%m-%d')
    else:
        return val


def gen_one(obj):
    for i in obj.__table__.columns:
        val = getattr(obj, i.name)
        if isinstance(val, datetime):
            yield i.name, val.strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(val, date):
            yield i.name, val.strftime('%Y-%m-%d')
        else:
            yield i.name, val
            
def gen_two(obj):
    for i in obj.__table__.columns:
        yield i.name, filter_func(getattr(obj, i.name))


Base = declarative_base()


class ObjectOne(Base):
    __tablename__ = 'object_one'
    id = Column(Integer, primary_key=True)
    first = Column(String)
    last = Column(String)
    created = Column(DateTime, default=func.now())
    update = Column(Date, default=func.now())

    def __init__(self, **kwargs):
        super(ObjectOne, self).__init__(**kwargs)

    @property
    def serialize(self):
        return {i.name: filter_func(getattr(self, i.name)) for i in self.__table__.columns}


class ObjectTwo(Base):
    __tablename__ = 'object_two'
    id = Column(Integer, primary_key=True)
    first = Column(String)
    last = Column(String)
    created = Column(DateTime, default=func.now())
    update = Column(Date, default=func.now())

    def __init__(self, **kwargs):
        super(ObjectTwo, self).__init__(**kwargs)

    @property
    def serialize(self):
        return {key: val for key, val in gen_one(self)}


class ObjectThree(Base):
    __tablename__ = 'object_three'
    id = Column(Integer, primary_key=True)
    first = Column(String)
    last = Column(String)
    created = Column(DateTime, default=func.now())
    update = Column(Date, default=func.now())

    def __init__(self, **kwargs):
        super(ObjectThree, self).__init__(**kwargs)

    @property
    def serialize(self):
        return {key: val for key, val in gen_two(self)}

In [16]:
a = ObjectOne(
    first='Ric',
    last='Flair',
    created=datetime.now(),
    update=date(2016, 1, 21)
)

print a.serialize

b = ObjectTwo(
    first='Ric',
    last='Flair',
    created=datetime.now(),
    update=date(2016, 1, 21)
)
print b.serialize

c = ObjectThree(
    first='Ric',
    last='Flair',
    created=datetime.now(),
    update=date(2016, 1, 21)
)
print c.serialize

{'update': '2016-01-21', 'created': '2016-08-16 19:40:02', 'last': 'Flair', 'id': None, 'first': 'Ric'}
{'update': '2016-01-21', 'created': '2016-08-16 19:40:02', 'last': 'Flair', 'id': None, 'first': 'Ric'}
{'update': '2016-01-21', 'created': '2016-08-16 19:40:02', 'last': 'Flair', 'id': None, 'first': 'Ric'}


In [23]:
%timeit -n 100000 -r 5 a.serialize

100000 loops, best of 5: 26.1 µs per loop


In [24]:
%timeit -n 100000 -r 5 b.serialize

100000 loops, best of 5: 27.8 µs per loop


In [25]:
%timeit -n 100000 -r 5 c.serialize

100000 loops, best of 5: 27.7 µs per loop


# Example
http://hastebin.com/paxiyaqinu.py